In [ ]:
#| echo: false
#| output: asis

from utils import get_recent_runs, load_run, get_most_recent_run
import json

# Load recent runs
recent_runs = get_recent_runs(limit=20)

# Get most recent run by default
selected_run_id = get_most_recent_run()
if selected_run_id:
    run_data = load_run(selected_run_id)
else:
    run_data = None

# Export for Observable
run_json = json.dumps(run_data, default=str) if run_data else "null"
runs_json = json.dumps(recent_runs, default=str)

::: {.grid}

::: {.g-col-12}
## Run Selector

Select a simulation run to explore:


```{ojs}
//| echo: false

// Load data from Python
runData = JSON.parse(`${run_json}`)
recentRuns = JSON.parse(`${runs_json}`)

// Run selector dropdown
viewof selectedRunId = Inputs.select(
  recentRuns.map(r => r.run_id),
  {
    label: "Simulation Run:",
    value: recentRuns[0]?.run_id,
    format: (id) => {
      const run = recentRuns.find(r => r.run_id === id);
      if (!run) return id;
      return `${run.template_id} (${run.started_at}) - $${run.cost_usd?.toFixed(2)}`;
    }
  }
)

// Load selected run
currentRun = {
  const run = recentRuns.find(r => r.run_id === selectedRunId);
  if (!run) return null;

  // Fetch full run data (would need API endpoint in production)
  // For now, use the default loaded run
  return selectedRunId === runData?.run_id ? runData : run;
}
```

:::

::: {.g-col-12 .g-col-md-6}
### 📊 Run Metadata


```{ojs}
//| echo: false

html`
<div class="metadata-card">
  <div class="metadata-row">
    <span class="label">Run ID:</span>
    <span class="value">${currentRun?.run_id || 'N/A'}</span>
  </div>
  <div class="metadata-row">
    <span class="label">Template:</span>
    <span class="value">${currentRun?.template_id || 'N/A'}</span>
  </div>
  <div class="metadata-row">
    <span class="label">Status:</span>
    <span class="value status-${currentRun?.status?.toLowerCase()}">${currentRun?.status || 'N/A'}</span>
  </div>
  <div class="metadata-row">
    <span class="label">Cost:</span>
    <span class="value">$${currentRun?.cost_usd?.toFixed(2) || '0.00'}</span>
  </div>
  <div class="metadata-row">
    <span class="label">Started:</span>
    <span class="value">${currentRun?.started_at || 'N/A'}</span>
  </div>
  <div class="metadata-row">
    <span class="label">Completed:</span>
    <span class="value">${currentRun?.completed_at || 'N/A'}</span>
  </div>
</div>
`
```

:::

::: {.g-col-12 .g-col-md-6}
### 🎯 Simulation Stats


```{ojs}
//| echo: false

html`
<div class="stats-card">
  <div class="stat-box">
    <div class="stat-value">${currentRun?.entities_created || 0}</div>
    <div class="stat-label">Entities</div>
  </div>
  <div class="stat-box">
    <div class="stat-value">${currentRun?.timepoints_created || 0}</div>
    <div class="stat-label">Timepoints</div>
  </div>
  <div class="stat-box">
    <div class="stat-value">${currentRun?.causal_mode || 'N/A'}</div>
    <div class="stat-label">Causal Mode</div>
  </div>
  <div class="stat-box">
    <div class="stat-value">${Object.keys(currentRun?.mechanisms_used || {}).length}</div>
    <div class="stat-label">Mechanisms</div>
  </div>
</div>
`
```

:::

::: {.g-col-12}
### 📝 Executive Summary


```{ojs}
//| echo: false

md`${currentRun?.narrative?.executive_summary || '*No summary available*'}`
```

:::

::: {.g-col-12}

---

## Visualizations

```{ojs}
//| echo: false

viewof activeTab = Inputs.radio(
  ["Timeline", "Network", "Metrics", "Screenplay"],
  {value: "Timeline"}
)
```
:::

::: {.g-col-12}
```{ojs}
//| echo: false

html`<div style="display: ${activeTab === 'Timeline' ? 'block' : 'none'}">${timelineViz}</div>`
html`<div style="display: ${activeTab === 'Network' ? 'block' : 'none'}">${networkViz}</div>`
html`<div style="display: ${activeTab === 'Metrics' ? 'block' : 'none'}">${metricsViz}</div>`
html`<div style="display: ${activeTab === 'Screenplay' ? 'block' : 'none'}">${screenplayViz}</div>`
```
:::

:::

```{ojs}
//| echo: false

// ========================================
// TIMELINE VISUALIZATION (vis-timeline)
// ========================================

import {Timeline} from "https://esm.sh/vis-timeline@7.7.3"
import "https://esm.sh/vis-timeline@7.7.3/styles/vis-timeline-graph2d.min.css"

timelineViz = {
  const container = html`<div style="height: 500px; border: 1px solid #ddd; border-radius: 8px;"></div>`;

  if (!currentRun?.narrative?.timepoints) {
    container.innerHTML = '<p style="padding: 20px;">No timepoint data available</p>';
    return container;
  }

  // Convert timepoints to vis-timeline items
  const items = currentRun.narrative.timepoints.map((tp, idx) => ({
    id: idx,
    content: tp.timepoint_id || `T${idx}`,
    start: tp.timestamp || new Date(),
    title: tp.event_description || '',
    type: 'point'
  }));

  const options = {
    height: '500px',
    margin: { item: 20 },
    orientation: 'top',
    showCurrentTime: false,
    zoomMin: 1000 * 60 * 60 * 24, // 1 day
    zoomMax: 1000 * 60 * 60 * 24 * 365 * 10 // 10 years
  };

  const timeline = new Timeline(container, items, options);

  return container;
}

// ========================================
// NETWORK VISUALIZATION (Cytoscape.js)
// ========================================

cytoscape = require("https://esm.sh/cytoscape@3.26.0")

networkViz = {
  const container = html`<div style="height: 600px; border: 1px solid #ddd; border-radius: 8px;"></div>`;

  if (!currentRun?.narrative?.characters) {
    container.innerHTML = '<p style="padding: 20px;">No character data available</p>';
    return container;
  }

  // Build nodes from characters
  const nodes = currentRun.narrative.characters.map(char => ({
    data: {
      id: char.entity_id,
      label: char.entity_id.replace(/_/g, ' '),
      type: char.entity_type || 'human'
    }
  }));

  // Build edges from relationships (if available)
  const edges = [];
  currentRun.narrative.characters.forEach(char => {
    if (char.relationships) {
      Object.entries(char.relationships).forEach(([target, rel]) => {
        edges.push({
          data: {
            source: char.entity_id,
            target: target,
            label: rel
          }
        });
      });
    }
  });

  const cy = cytoscape({
    container: container,
    elements: [...nodes, ...edges],
    style: [
      {
        selector: 'node',
        style: {
          'background-color': '#667',
          'label': 'data(label)',
          'text-valign': 'center',
          'text-halign': 'center',
          'font-size': '12px',
          'width': 50,
          'height': 50
        }
      },
      {
        selector: 'edge',
        style: {
          'width': 2,
          'line-color': '#999',
          'target-arrow-color': '#999',
          'target-arrow-shape': 'triangle',
          'curve-style': 'bezier',
          'label': 'data(label)',
          'font-size': '10px'
        }
      }
    ],
    layout: {
      name: 'cose',
      animate: true,
      animationDuration: 1000,
      idealEdgeLength: 100,
      nodeOverlap: 20
    }
  });

  return container;
}

// ========================================
// METRICS VISUALIZATION (ECharts)
// ========================================

echarts = require("https://esm.sh/echarts@5.4.3")

metricsViz = {
  const container = html`<div style="height: 600px;"></div>`;

  const chart = echarts.init(container);

  // Mechanism usage radar chart
  const mechanisms = currentRun?.mechanisms_used || {};
  const mechanismNames = Object.keys(mechanisms);
  const mechanismValues = Object.values(mechanisms);

  const option = {
    title: {
      text: 'Mechanism Coverage',
      left: 'center',
      top: 20,
      textStyle: {
        fontSize: 20,
        fontWeight: 'bold'
      }
    },
    tooltip: {
      trigger: 'item'
    },
    radar: {
      indicator: mechanismNames.map(name => ({
        name: name,
        max: Math.max(...mechanismValues) || 10
      })),
      radius: '60%',
      center: ['50%', '55%']
    },
    series: [{
      name: 'Mechanism Usage',
      type: 'radar',
      data: [{
        value: mechanismValues,
        name: 'Usage Count',
        areaStyle: {
          color: 'rgba(99, 102, 241, 0.3)'
        },
        lineStyle: {
          color: 'rgba(99, 102, 241, 1)',
          width: 2
        },
        itemStyle: {
          color: 'rgba(99, 102, 241, 1)'
        }
      }]
    }]
  };

  chart.setOption(option);

  return container;
}

// ========================================
// SCREENPLAY VIEWER (Stubbed)
// ========================================

screenplayViz = {
  const container = html`<div class="screenplay-stub"></div>`;

  if (!currentRun?.has_screenplay) {
    container.innerHTML = `
      <div style="padding: 40px; text-align: center; color: #666;">
        <h3>📝 Screenplay Viewer</h3>
        <p>No screenplay available for this run.</p>
        <p><em>TODO: Implement Fountain screenplay parser and renderer</em></p>
      </div>
    `;
  } else {
    container.innerHTML = `
      <div style="padding: 40px; text-align: center; color: #666;">
        <h3>📝 Screenplay Viewer</h3>
        <p>Screenplay available but viewer not yet implemented.</p>
        <p><em>TODO: Integrate Fountain.js or similar parser</em></p>
        <details>
          <summary>Show raw Fountain content</summary>
          <pre style="text-align: left; max-height: 400px; overflow: auto; padding: 20px; background: #f5f5f5; border-radius: 8px;">
${currentRun.screenplay?.substring(0, 1000) || 'N/A'}...
          </pre>
        </details>
      </div>
    `;
  }

  return container;
}
```

---


## 🚧 Advanced Features (Coming Soon)

The following features are planned for future implementation:

### PORTAL Mode Visualization
- Backward causality arrows showing path from portal to origin
- Pivot point highlighting with alternate timeline branches
- Plausibility score heatmap for candidate paths

### Run Comparison
- Side-by-side timeline comparison for 2+ runs
- Diff view showing divergence points
- Cost and mechanism usage comparison charts

### Search & Filter
- Full-text search across entity knowledge and event descriptions
- Filter timepoints by date range, mechanism, or entity participation
- Export filtered datasets to JSON/CSV

### Interactive Exports
- PNG/SVG export of visualizations
- PDF report generation with embedded charts
- Shareable dashboard links